In [ ]:
import numpy as np
import panel as pn

pn.extension()

One of the main design goals for Panel was that it should make it possible to seamlessly transition back and forth between interactively prototyping a dashboard in the notebook or on the commandline to deploying it as a standalone server app. This section shows how to display panels interactively, embed static output, save a snapshot, and deploy as a separate web-server app. For more information about deploying Panel apps to various cloud providers see the [Server Deployment](Server_Deployment.ipynb) documentation.

## Configuring output

As you may have noticed, almost all the Panel documentation is written using notebooks. Panel objects display themselves automatically in a notebook and take advantage of Jupyter Comms to support communication between the rendered app and the Jupyter kernel that backs it on the Python end. To display a Panel object in the notebook is as simple as putting it on the end of a cell. Note, however, that the ``panel.extension`` first has to be loaded to initialize the required JavaScript in the notebook context. In recent versions of JupyterLab this works out of the box but for older versions (`<3.0`) the PyViz labextension has to be installed with:

    jupyter labextension install @pyviz/jupyterlab_pyviz

### Optional dependencies

Also remember that in order to use certain components such as Vega, LaTeX, and Plotly plots in a notebook, the models must be loaded using the extension. If you forget to load the extension, you should get a warning reminding you to do it. To load certain JS components, simply list them as part of the call to ``pn.extension``:

    pn.extension('vega', 'katex')

Here we've ensured that the Vega and LaTeX JS dependencies will be loaded.

### Initializing JS and CSS 

Additionally, any external ``css_files``, ``js_files`` and ``raw_css`` needed should be declared in the extension. The ``js_files`` should be declared as a dictionary mapping from the exported JS module name to the URL containing the JS components, while the ``css_files`` can be defined as a list:

    pn.extension(js_files={'deck': https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js},
                 css_files=['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css'])

The ``raw_css`` argument allows defining a list of strings containing CSS to publish as part of the notebook and app.

Providing keyword arguments via the ``extension`` is the same as setting them on ``pn.config``, which is the preferred approach outside the notebook. ``js_files`` and ``css_files`` may be set to your chosen values as follows:

    pn.config.js_files  = {'deck': 'https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js'}
    pn.config.css_files = ['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css']

## Display in the notebook

#### The repr
    
Once the extension is loaded, Panel objects will display themselves if placed at the end of cell in the notebook:

In [ ]:
pane = pn.panel('<marquee>Here is some custom HTML</marquee>')

pane

To instead see a textual representation of the component, you can use the ``pprint`` method on any Panel object:

In [ ]:
pane.pprint()

#### The ``display`` function

To avoid having to put a Panel on the last line of a notebook cell, e.g. to display it from inside a function call, you can use the IPython built-in ``display`` function:

In [ ]:
def display_marquee(text):
    display(pn.panel('<marquee>{text}</marquee>'.format(text=text)))
    
display_marquee('This Panel was displayed from within a function')

#### Inline apps

Lastly it is also possible to display a Panel object as a Bokeh server app inside the notebook.  To do so call the ``.app`` method on the Panel object and provide the URL of your notebook server:

In [ ]:
pane.app('localhost:8888')

The app will now run on a Bokeh server instance separate from the Jupyter notebook kernel, allowing you to quickly test that all the functionality of your app works both in a notebook and in a server context.

### ipywidgets

If the `jupyter_bokeh` package is installed it is also possible to render Panel objects as an ipywidget rather than using Bokeh's internal communication mechanisms.  You can enable ipywidgets support  globally using:

```python
pn.extension(comms='ipywidgets')
# or
pn.config.comms = 'ipywidgets'
```

This global setting can be useful when trying to serve an entire notebook using [Voilà](https://github.com/voila-dashboards/voila). Alternatively, we can convert individual objects to an ipywidget one at a time using the `pn.ipywidget()` function:

```python
ipywidget = pn.ipywidget(pane)
ipywidget
```

This approach also allows combining a Panel object with any other Jupyter-widget--based model:

```python
from ipywidgets import Accordion
Accordion(children=[pn.ipywidget(pane)])
```

To use Panel's ipywidgets support in JupyterLab, the following extensions have to be installed:
    
```
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install @bokeh/jupyter_bokeh
```

Additionally the `jupyter_bokeh` package should be installed using either pip:

```
pip install jupyter_bokeh
```

or using conda:

```
conda install -c bokeh jupyter_bokeh
```

## Launching a server dynamically

The CLI `panel serve` command described below is usually the best approach for deploying applications. However when working on the REPL or embedding a Panel/Bokeh server in another application it is sometimes useful to dynamically launch a server, either using the `.show` method or using the `pn.serve` function.

### Previewing an application

Working from the command line will not automatically display rich representations inline as in a notebook, but you can still interact with your Panel components if you start a Bokeh server instance and open a separate browser window using the ``show`` method. The method has the following arguments:

    port: int (optional)
       Allows specifying a specific port (default=0 chooses an arbitrary open port)
    websocket_origin: str or list(str) (optional)
       A list of hosts that can connect to the websocket.
       This is typically required when embedding a server app in
       an external-facing web site.
       If None, "localhost" is used.
    threaded: boolean (optional, default=False)
       Whether to launch the Server on a separate thread, allowing
       interactive use.
    title : str
       A string title to give the Document (if served as an app)
    **kwargs : dict
       Additional keyword arguments passed to the bokeh.server.server.Server instance.

To work with an app completely interactively you can set ``threaded=True`` which will launch the server on a separate thread and let you interactively play with the app.

<img src='https://assets.holoviews.org/panel/gifs/commandline_show.gif'></img>

The ``.show`` call will return either a Bokeh server instance (if ``threaded=False``) or a ``StoppableThread`` instance (if ``threaded=True``) which both provide a ``stop`` method to stop the server instance.

### Serving multiple apps

If you want to serve more than one app on a single server you can use the ``pn.serve`` function. By supplying a dictionary where the keys represent the URL slugs and the values must be either Panel objects or functions returning Panel objects you can easily launch a server with a number of apps, e.g.:

```python
pn.serve({
    'markdown': '# This is a Panel app',
    'json': pn.pane.JSON({'abc': 123})
})
```

Note that when you serve an object directly all sessions will share the same state, i.e. the parameters of all components will be synced across sessions such that the change in a widget by one user will affect all other users. Therefore you will usually want to wrap your app in a function, ensuring that each user gets a new instance of the application:

```python

def markdown_app():
    return '# This is a Panel app'

def json_app():
    return pn.pane.JSON({'abc': 123})

pn.serve({
    'markdown': markdown_app,
    'json': json_app
})
```

You can customize the HTML title of each application by supplying a dictionary where the keys represent the URL slugs and the values represent the titles, e.g.:

```python
pn.serve({
    'markdown': '# This is a Panel app',
    'json': pn.pane.JSON({'abc': 123})
}, title={'markdown': 'A Markdown App', 'json': 'A JSON App'}
)
```

The ``pn.serve`` accepts a number of arguments:

    panel: Viewable, function or {str: Viewable or function}
      A Panel object, a function returning a Panel object or a
      dictionary mapping from the URL slug to either.
    port: int (optional, default=0)
      Allows specifying a specific port
    address: str
      The address the server should listen on for HTTP requests.
    websocket_origin: str or list(str) (optional)
      A list of hosts that can connect to the websocket.

      This is typically required when embedding a server app in
      an external web site.

      If None, "localhost" is used.
    loop: tornado.ioloop.IOLoop (optional, default=IOLoop.current())
      The tornado IOLoop to run the Server on
    show: boolean (optional, default=False)
      Whether to open the server in a new browser tab on start
    start: boolean(optional, default=False)
      Whether to start the Server
    title: str or {str: str} (optional, default=None)
      An HTML title for the application or a dictionary mapping
      from the URL slug to a customized title
    verbose: boolean (optional, default=True)
      Whether to print the address and port
    location: boolean or panel.io.location.Location
      Whether to create a Location component to observe and
      set the URL location.
    kwargs: dict
      Additional keyword arguments to pass to Server instance

## Launching a server on the commandline

Once the app is ready for deployment it can be served using the Bokeh server.  For a detailed breakdown of the design and functionality of Bokeh server, see the [Bokeh documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/server.html). The most important thing to know is that Panel (and Bokeh) provide a CLI command to serve a Python script, app directory, or Jupyter notebook containing a Bokeh or Panel app. To launch a server using the CLI, simply run:

    panel serve app.ipynb
    
Alternatively you can also list multiple apps:

    panel serve app1.py app2.ipynb
    
or even serve a number of apps at once:

    panel serve apps/*.py
    
For development it can be particularly helpful to use the ``--autoreload`` option to `panel serve` as that will automatically reload the page whenever the application code or any of its imports change.
    
The ``panel serve`` command has the following options:

    positional arguments:
      DIRECTORY-OR-SCRIPT   The app directories or scripts or notebooks to serve 
                            (serve empty document if not specified)

    optional arguments:
      -h, --help            show this help message and exit
      --port PORT           Port to listen on
      --address ADDRESS     Address to listen on
      --log-level LOG-LEVEL
                            One of: trace, debug, info, warning, error or critical
      --log-format LOG-FORMAT
                            A standard Python logging format string (default:
                            '%(asctime)s %(message)s')
      --log-file LOG-FILE   A filename to write logs to, or None to write to the
                            standard stream (default: None)
      --args ...            Any command line arguments remaining are passed on to
                            the application handler
      --show                Open server app(s) in a browser
      --allow-websocket-origin HOST[:PORT]
                            Public hostnames which may connect to the Bokeh
                            websocket
      --prefix PREFIX       URL prefix for Bokeh server URLs
      --keep-alive MILLISECONDS
                            How often to send a keep-alive ping to clients, 0 to
                            disable.
      --check-unused-sessions MILLISECONDS
                            How often to check for unused sessions
      --unused-session-lifetime MILLISECONDS
                            How long unused sessions last
      --stats-log-frequency MILLISECONDS
                            How often to log stats
      --mem-log-frequency MILLISECONDS
                            How often to log memory usage information
      --use-xheaders        Prefer X-headers for IP/protocol information
      --auth-module AUTH_MODULE
                            Absolute path to a Python module that implements auth hooks
      --enable-xsrf-cookies
                            Whether to enable Tornado support for XSRF cookies.
                            All PUT, POST, or DELETE handlers must be properly
                            instrumented when this setting is enabled.
      --exclude-headers EXCLUDE_HEADERS [EXCLUDE_HEADERS ...]
                            A list of request headers to exclude from the session
                            context (by default all headers are included).
      --exclude-cookies EXCLUDE_COOKIES [EXCLUDE_COOKIES ...]
                            A list of request cookies to exclude from the session
                            context (by default all cookies are included).
      --include-headers INCLUDE_HEADERS [INCLUDE_HEADERS ...]
                            A list of request headers to make available in the
                            session context (by default all headers are included).
      --include-cookies INCLUDE_COOKIES [INCLUDE_COOKIES ...]
                            A list of request cookies to make available in the
                            session context (by default all cookies are included).
      --session-ids MODE    One of: unsigned, signed, or external-signed
      --index INDEX         Path to a template to use for the site index
      --disable-index       Do not use the default index on the root path
      --disable-index-redirect
                            Do not redirect to running app from root path
      --default-app FILENAME
                            The app to redirect to from the root. When enabled
                            no index is served.
      --num-procs N         Number of worker processes for an app. Using 0 will
                            autodetect number of cores (defaults to 1)
      --warm                Whether to execute scripts on startup to warm up the server.
      --autoreload
                            Whether to automatically reload user sessions when the application or any of its imports change.
      --static-dirs KEY=VALUE [KEY=VALUE ...]        
                            Static directories to serve specified as key=value
                            pairs mapping from URL route to static file directory.

      --dev [FILES-TO-WATCH [FILES-TO-WATCH ...]]
                            Enable live reloading during app development.By
                            default it watches all *.py *.html *.css *.yaml
                            filesin the app directory tree. Additional files can
                            be passedas arguments. NOTE: This setting only works
                            with a single app.It also restricts the number of
                            processes to 1.
      --session-token-expiration N
                            Duration in seconds that a new session token is valid
                            for session creation. After the expiry time has elapsed,
                            the token will not be able create a new session
                            (defaults to seconds).
      --websocket-max-message-size BYTES
                            Set the Tornado websocket_max_message_size value
                            (defaults to 20MB) NOTE: This setting has effect ONLY
                            for Tornado>=4.5
      --websocket-compression-level LEVEL
                            Set the Tornado WebSocket compression_level
      --websocket-compression-mem-level LEVEL
                            Set the Tornado WebSocket compression mem_level
      --oauth-provider OAUTH_PROVIDER
                            The OAuth2 provider to use.
      --oauth-key OAUTH_KEY
                            The OAuth2 key to use
      --oauth-secret OAUTH_SECRET
                            The OAuth2 secret to use
      --oauth-redirect-uri OAUTH_REDIRECT_URI
                            The OAuth2 redirect URI
      --oauth-extra-params OAUTH_EXTRA_PARAMS
                            Additional parameters to use.
      --oauth-jwt-user OAUTH_JWT_USER
                            The key in the ID JWT token to consider the user.
      --oauth-encryption-key OAUTH_ENCRYPTION_KEY
                            A random string used to encode the user information.
      --rest-provider REST_PROVIDER
                            The interface to use to serve REST API
      --rest-endpoint REST_ENDPOINT
                            Endpoint to store REST API on.
      --rest-session-info   
                            Whether to serve session info on the REST API
      --session-history SESSION_HISTORY
                            The length of the session history to record.

To turn a notebook into a deployable app simply append ``.servable()`` to one or more Panel objects, which will add the app to Bokeh's ``curdoc``, ensuring it can be discovered by Bokeh server on deployment. In this way it is trivial to build dashboards that can be used interactively in a notebook and then seamlessly deployed on Bokeh server.

When called on a notebook, `panel serve` first converts it to a python script using [`nbconvert.PythonExporter()`](https://nbconvert.readthedocs.io/en/stable/api/exporters.html), albeit with [IPython magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) stripped out. This means that non-code cells, such as raw cells, are entirely handled by `nbconvert` and [may modify the served app](https://nbsphinx.readthedocs.io/en/latest/raw-cells.html).

## Static file hosting

Whether you're launching your application using `panel serve` from the commandline or using `pn.serve` in a script you can also serve static files. When using `panel serve` you can use the `--static-dirs` argument to specify a list of static directories to serve along with their routes, e.g.:

    panel serve some_script.py --static-dirs assets=./assets
    
This will serve the `./assets` directory on the servers `/assets` route. Note however that the `/static` route is reserved internally by Panel.

Similarly when using `pn.serve` or `panel_obj.show` the static routes may be defined as a dictionary, e.g. the equivalent to the example would be:

    pn.serve(panel_obj, static_dirs={'assets': './assets'})

## Accessing session state

Whenever a Panel app is being served the ``panel.state`` object exposes some of the internal Bokeh server components to a user.

#### Document

The current Bokeh ``Document`` can be accessed using ``panel.state.curdoc``.

#### Request arguments

When a browser makes a request to a Bokeh server a session is created for the Panel application. The request arguments are made available to be accessed on ``pn.state.session_args``. For example if your application is hosted at ``localhost:8001/app``, appending ``?phase=0.5`` to the URL will allow you to access the phase variable using the following code:

```python
try:
    phase = int(pn.state.session_args.get('phase')[0])
except Exception:
    phase = 1
```

This mechanism may be used to modify the behavior of an app dependending on parameters provided in the URL. 

#### Cookies

The `panel.state.cookies` will allow accessing the cookies stored in the browser and on the bokeh server.

#### Headers

The `panel.state.headers` will allow accessing the HTTP headers stored in the browser and on the bokeh server.

#### Location

When starting a server session Panel will attach a `Location` component which can be accessed using `pn.state.location`. The `Location` component servers a number of functions:

- Navigation between pages via ``pathname``
- Sharing (parts of) the page state in the url as ``search`` parameters for bookmarking and sharing.
- Navigating to subsections of the page via the ``hash_`` parameter.

##### Core

* **``pathname``** (string): pathname part of the url, e.g. '/user_guide/Interact.html'.
* **``search``** (string): search part of the url e.g. '?color=blue'.
* **``hash_``** (string): hash part of the url e.g. '#interact'.
* **``reload``** (bool): Whether or not to reload the page when the url is updated.
    - For independent apps this should be set to True. 
    - For integrated or single page apps this should be set to False.

##### Readonly

* **``href``** (string): The full url, e.g. 'https://panel.holoviz.org/user_guide/Interact.html:80?color=blue#interact'.
* **``protocol``** (string): protocol part of the url, e.g. 'http:' or 'https:'
* **``port``** (string): port number, e.g. '80'

#### pn.state.busy

Often an application will have longer running callbacks which are being processed on the server, to give users some indication that the server is busy you may therefore have some way of indicating that busy state. The `pn.state.busy` parameter indicates whether a callback is being actively processed and may be linked to some visual indicator.

Below we will create a little application to demonstrate this, we will create a button which executes some longer running task on click and then create an indicator function that displays `'I'm busy'` when the `pn.state.busy` parameter is `True` and `'I'm idle'` when it is not:

In [ ]:
import time

def processing(event):
    # Some longer running task
    time.sleep(1)
    
button = pn.widgets.Button(name='Click me!')
button.on_click(processing)

@pn.depends(pn.state.param.busy)
def indicator(busy):
    return "I'm busy" if busy else "I'm idle"

pn.Row(button, indicator)

This way we can create a global indicator for the busy state instead of modifying all our callbacks.

#### pn.state.onload

Another useful callback to define the onload callback, in a server context this will execute when a session is first initialized. Let us for example define a minimal example inside a function which we will pass to `pn.serve`. This emulates what happens when we call `panel serve` on the commandline. We will create a widget without populating its options, then we will add an `onload` callback, which will set the options once the initial page is loaded. Imagine for example that we have to fetch the options from some database which might take a little while, by deferring the loading of the options to the callback we can get something on the screen as quickly as possible and only run the expensive callback when we have already rendered something for the user to look at.

In [ ]:
import time

def app():
    widget = pn.widgets.Select()

    def on_load():
        time.sleep(1) # Emulate some long running process
        widget.options = ['A', 'B', 'C']

    pn.state.onload(on_load)

    return widget

# pn.serve(app) 

#### pn.state.on_session_destroyed

In many cases it is useful to define on_session_destroyed callbacks to perform any custom cleanup that is required, e.g,  dispose  a database engine, or when a user is logged out. These callbacks can be registered with `pn.state.on_session_destroyed(callback)`

## Scheduling callbacks

When you build an app you frequently want to schedule a callback to be run periodically to refresh the data and update visual components. Additionally if you want to update Bokeh components directly you may need to schedule a callback to get around Bokeh's document lock to avoid errors like this:

> RuntimeError: _pending_writes should be non-None when we have a document lock, and we should have the lock when the document changes

In this section we will discover how we can leverage Bokeh's Document and `pn.state.add_periodic_callback` to set this up.

### Server callbacks

The Bokeh server that Panel builds on is designed to be thread safe which requires a set of locks to avoid multiple threads modifying the Bokeh models simultaneously. Therefore if we want to work with Bokeh models directly we should ensure that any changes to a Bokeh model are executed on the correct thread by adding a callback, which the event loop will then execute safely.

In the example below we will launch an application on a thread using `pn.serve` and make the Bokeh plot (in practice you may provide handles to this object on a class). Finally we will wait 1 second until the server is launched and schedule a callback which updates the `y_range` by accessing the `Document` and calling `add_next_tick_callback` on it. This pattern will ensure that the update to the Bokeh model is executed on the correct thread:

```python
import time
import panel as pn

from bokeh.plotting import figure

global p
p = None

def app():
    global p
    doc = pn.state.curdoc
    p = figure()
    p.line([1, 2, 3], [1, 2, 3])
    return p

pn.serve(app, threaded=True)

time.sleep(1)

p.document.add_next_tick_callback(lambda: p.y_range.update(start=0, end=4))
```

### Periodic callbacks

As we discussed above periodic callbacks allow periodically updating your application with new data. Below we will create a simple Bokeh plot and display it with Panel:

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

source = ColumnDataSource({"x": range(10), "y": range(10)})
p = figure()
p.line(x="x", y="y", source=source)

bokeh_pane = pn.pane.Bokeh(p)
bokeh_pane.servable()

Now we will define a callback that updates the data on the `ColumnDataSource` and use the `pn.state.add_periodic_callback` method to schedule updates every 200 ms. We will also set a timeout of 5 seconds after which the callback will automatically stop.

In [ ]:
def update():
    data = np.random.randint(0, 2 ** 31, 10)
    source.data.update({"y": data})
    bokeh_pane.param.trigger('object') # Only needed in notebook

cb = pn.state.add_periodic_callback(update, 200, timeout=5000)

In a notebook or bokeh server context we should now see the plot update periodically. The other nice thing about this is that `pn.state.add_periodic_callback` returns `PeriodicCallback` we can call `.stop()` and `.start()` on if we want to stop or pause the periodic execution. Additionally we can also dynamically adjust the period by setting the `timeout` parameter to speed up or slow down the callback.

Other nice features on a periodic callback are the ability to check the number of executions using the `cb.counter` property and the ability to toggle the callback on and off simply by setting the running parameter. This makes it possible to link a widget to the running state:

In [ ]:
toggle = pn.widgets.Toggle(name='Toggle callback')

toggle.link(cb, bidirectional=True, value='running')
toggle

Note that when starting a server dynamically with `pn.serve` you cannot start a periodic callback before the application is actually being served. Therefore you should create the application and start the callback in a wrapping function:

```python
from functools import partial

import numpy as np
import panel as pn

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

def update(source):
    data = np.random.randint(0, 2 ** 31, 10)
    source.data.update({"y": data})

def panel_app():
    source = ColumnDataSource({"x": range(10), "y": range(10)})
    p = figure()
    p.line(x="x", y="y", source=source)
    cb = pn.state.add_periodic_callback(partial(update, source), 200, timeout=5000)
    return pn.pane.Bokeh(p)

pn.serve(panel_app)
```

## Accessing the Bokeh model

Since Panel is built on top of Bokeh, all Panel objects can easily be converted to a Bokeh model. The ``get_root`` method returns a model representing the contents of a Panel:

In [ ]:
model = pn.Column('# Some markdown').get_root()
model

By default this model will be associated with Bokeh's ``curdoc()``, so if you want to associate the model with some other ``Document`` ensure you supply it explictly as the first argument. Once you have access to the underlying bokeh model you can use all the usual bokeh utilities such as ``components``, ``file_html``, or ``show``

In [ ]:
from bokeh.embed import components, file_html
from bokeh.io import show

script, html = components(model)

## Embedding

Panel generally relies on either the Jupyter kernel or a Bokeh Server to be running in the background to provide interactive behavior. However for simple apps with a limited amount of state it is also possible to `embed` all the widget state, allowing the app to be used entirely from within Javascript. To demonstrate this we will create a simple app which simply takes a slider value, multiplies it by 5 and then display the result.

In [ ]:
slider = pn.widgets.IntSlider(start=0, end=10)

@pn.depends(slider.param.value)
def callback(value):
    return '%d * 5 = %d' % (value, value*5)

row = pn.Row(slider, callback)

If we displayed this the normal way it would call back into Python every time the value changed. However, the `.embed()` method will record the state of the app for the different widget configurations.

In [ ]:
row.embed()

If you try the widget above you will note that it only has 3 different states, 0, 5 and 10. This is because by default embed will try to limit the number of options of non-discrete or semi-discrete widgets to at most three values. This can be controlled using the `max_opts` argument to the embed method or you can provide an explicit list of `states` to embed for each widget:

In [ ]:
row.embed(states={slider: list(range(0, 12, 2))})

 The full set of options for the embed method include:

- **`max_states`**: The maximum number of states to embed

- **`max_opts`**: The maximum number of states for a single widget

- **`states`** (default={}): A dictionary specifying the widget values to embed for each widget

- **`json`** (default=True): Whether to export the data to json files

- **`save_path`** (default='./'): The path to save json files to

- **`load_path`** (default=None):  The path or URL the json files will be loaded from (same as ``save_path`` if not specified)

* **`progress`** (default=False): Whether to report progress


As you might imagine if there are multiple widgets there can quickly be a combinatorial explosion of states so by default the output is limited to about 1000 states. For larger apps the states can also be exported to json files, e.g. if you want to serve the app on a website specify the ``save_path`` to declare where it will be stored and the ``load_path`` to declare where the JS code running on the website will look for the files.

## Saving 

In case you don't need an actual server or simply want to export a static snapshot of a panel app, you can use the ``save`` method, which allows exporting the app to a standalone HTML or PNG file.

By default, the HTML file generated will depend on loading JavaScript code for BokehJS from the online ``CDN`` repository, to reduce the file size. If you need to work in an airgapped or no-network environment, you can declare that ``INLINE`` resources should be used instead of ``CDN``:

```python
from bokeh.resources import INLINE
panel.save('test.html', resources=INLINE)
```

Additionally the save method also allows enabling the `embed` option, which, as explained above, will embed the apps state in the app or save the state to json files which you can ship alongside the exported HTML.

Finally, if a 'png' file extension is specified, the exported plot will be rendered as a PNG, which currently requires Selenium and PhantomJS to be installed:

```python
pane.save('test.png')

```